In [13]:
import numpy as np
import pandas as pd # data processing, CSV file

import os
for dirname, _, filenames in os.walk('D:\BART\Bart\coin_data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:\BART\Bart\coin_data\coin_Aave.csv
D:\BART\Bart\coin_data\coin_BinanceCoin.csv
D:\BART\Bart\coin_data\coin_Bitcoin.csv
D:\BART\Bart\coin_data\coin_Cardano.csv
D:\BART\Bart\coin_data\coin_ChainLink.csv
D:\BART\Bart\coin_data\coin_Cosmos.csv
D:\BART\Bart\coin_data\coin_CryptocomCoin.csv
D:\BART\Bart\coin_data\coin_Dogecoin.csv
D:\BART\Bart\coin_data\coin_EOS.csv
D:\BART\Bart\coin_data\coin_Ethereum.csv
D:\BART\Bart\coin_data\coin_Iota.csv
D:\BART\Bart\coin_data\coin_Litecoin.csv
D:\BART\Bart\coin_data\coin_Monero.csv
D:\BART\Bart\coin_data\coin_NEM.csv
D:\BART\Bart\coin_data\coin_Polkadot.csv
D:\BART\Bart\coin_data\coin_Solana.csv
D:\BART\Bart\coin_data\coin_Stellar.csv
D:\BART\Bart\coin_data\coin_Tether.csv
D:\BART\Bart\coin_data\coin_Tron.csv
D:\BART\Bart\coin_data\coin_Uniswap.csv
D:\BART\Bart\coin_data\coin_USDCoin.csv
D:\BART\Bart\coin_data\coin_WrappedBitcoin.csv
D:\BART\Bart\coin_data\coin_XRP.csv


In [28]:
import csv

# Specify the path to your CSV file
csv_file_path = "D:\BART\Bart\coin_data\coin_Dogecoin.csv"

# Open the CSV file in read mode
with open(csv_file_path, 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    
    # Read the header row
    header = next(csv_reader)
    
    # Print the column names
    print("Columns in the CSV file:")
    for column in header:
        print(column)


Columns in the CSV file:
SNo
Name
Symbol
Date
High
Low
Open
Close
Volume
Marketcap


In [27]:
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999 #Outputs more colums for dataframes

base = "D:\BART\Bart\coin_data"
datahistory = 60 #Amount of days for data history to use

def coincount(data_drt): # counts amount of coin files within the given directory

    coin__Count = 0
    for cryptofile in os.listdir(data_drt):
        if cryptofile.endswith(".csv"):
            coin__Count += 1
    return coin__Count


def get_max_min_data_lengths(data_drt):
    #It finds min and max lengths of the data coin files.
    lengthmax = 0
    lengthmin = float("inf")  #Actiavtes lengthmin to be infinity
    for cryptofile in os.listdir(data_drt):
        if cryptofile.endswith(".csv"):
            fD = pd.read_csv(data_drt + "/" + cryptofile, parse_dates=['Date'])
            lengthdata = fD.shape[0]
            lengthmax = max(lengthmax, lengthdata)
            lengthmin = min(lengthmin, lengthdata)
    return lengthmax,lengthmin


def read_process_coin_data(data_drt, indexCoin):
    # reads and processes a specific coin
    cryptofile = os.listdir(data_drt)[indexCoin]
    fD = pd.read_csv(data_drt + "/" + cryptofile, parse_dates=['Date'])
    symbolCoin = cryptofile[5:-4]
    lengthdata = fD.shape[0]
    pricesclosing = fD['closed'].values
    return symbolCoin, lengthdata, pricesclosing


def read_data(data_drt, datahistory):
    # reads and processes the cryptocurrency data
    coinNum = coin__Count(data_drt)
    maxlengthdata = get_max_min_data_lengths(data_drt)  #length_min discarded
    pricesclosingdata = np.zeros((coinNum, maxlengthdata))
    coinlengthsdata = np.zeros(coinNum, dtype=int)


    for indexCoin in range(coinNum):
        symbolCoin, lengthdata, pricesclosing = read_process_coin_data(data_drt, indexCoin)
        print(indexCoin, symbolCoin, lengthdata)
        priceslosingdata[indexCoin, 0:lengthdata] = pricesclosing
        coinlengthsdata[indexCoin] = lengthdata


def read_data (): # Reading and processing all data of coins

    coinNum = 0
    for name in os.listdir(base):
        coinNum += 1 # counts the amount of files with coins

    lengthmax, lengthmin = 0, 1000000 # intiating a big value
    for name in os.listdir(base):
        fD = pd.read_csv(base + "/" + name, parse_dates=['Date'])
        thelength = fD.shape[0]
        if lengthmax < thelength:
            lengthmax = thelength
        if lengthmin > thelength:
            lengthmin = thelength


    info = np.zeros ((coinNum, lengthmax)) #Activates arrays to keep coins data
    Slength = np.zeros(coinNum, dtype = int)
    i = 0
    for name in os.listdir(base):
        coinSymb = name[5:-4] #Extracting the symbol of the coin from the file name.
        fD = pd.read_csv(base + "/" + name, parse_dates=['Date'])
        thelength = fD.shape[0]

        #Storing coin info.
        Slength[i] = thelength
        print (i, coinSymb, thelength)

        info[i, 0:thelength] = fD['Close'].values
        i += 1


    return coinNum, Slength, info

coinNum, Slength, info = read_data ()
print ("Coin amount: ", coinNum, ".")

0 Aave 275
1 BinanceCoin 1442
2 Bitcoin 2991
3 Cardano 1374
4 ChainLink 1385
5 Cosmos 845
6 CryptocomCoin 935
7 Dogecoin 2760
8 EOS 1466
9 Ethereum 2160
10 Iota 1484
11 Litecoin 2991
12 Monero 2602
13 NEM 2288
14 Polkadot 320
15 Solana 452
16 Stellar 2527
17 Tether 2318
18 Tron 1392
19 Uniswap 292
20 USDCoin 1002
21 WrappedBitcoin 888
22 XRP 2893
Coin amount:  23 .


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [15]:
def data_scale(info, Slength):

    coinNum = info.shape[0] #More cleaner name for the variable.
    shift_info = np.zeros(coinNum)
    factor_scale = np.zeros(coinNum)
    #^^^ shift_info and factor_scale have descriptive name.

    for i in range (coinNum):
        valMax = info[i,:Slength[i]].max()
        valMin = info[i, :Slength[i]].min()
        #^^^ Clearing calculation ^^^
        shift_info[i] = valMin #This stores the info of shift value of the info.
        factor_scale[i] = valMax - valMin #Calculating the factor scale.

        if factor_scale[i] == 0:
            raise ValueError("Division by zero encountered during scaling.")

        info[i,0:Slength[i]] = (info[i,0:Slength[i]]-shift_info[i])/factor_scale[i]
    return (shift_info, factor_scale) #Scaling the info and scale factor

shift_info, factor_scale = data_scale (info, Slength)

In [16]:
def sequence(info, Slength, begin, finish, datahistory):
    assert len(info) == len(Slength), "Unmatched data length."

    # Initialize blank lists to keep targets and sequences
    A = []
    B = []

    # Iterate over the specified range of coins
    for j in range(begin, finish):
        for i in range(datahistory, Slength[j]):
            # Append input sequence and its compatible value target
            A.append(info[j, i - datahistory:i])
            B.append(info[j, i])

    X = np.array(A)[:, :, np.newaxis] # Converting lists to numpy arrays and reshape for LSTM input
    y = np.array(B)

    return X, y

# number of days for data history to use ie 15 days
datahistory = 15

# Creating the validation and training, and test sequences
X_val, Y_val = sequence(info, Slength, 18, 22, datahistory)
print("Approximately", Y_val.shape[0], "sequences for validation.")

X_test, Y_test = sequence(info, Slength, 22, 23, datahistory)
print("Approximately", Y_test.shape[0], "sequences for testing.")

X_train, Y_train = sequence(info, Slength, 0, 18, datahistory)
print("Approximately", Y_train.shape[0], "sequences for training.")


Approximately 3514 sequences for validation.
Approximately 2878 sequences for testing.
Approximately 30345 sequences for training.


In [22]:
import numpy as np

class LSTMModel:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.W_f = np.random.randn(hidden_size + input_size, hidden_size)
        self.W_i = np.random.randn(hidden_size + input_size, hidden_size)
        self.W_c = np.random.randn(hidden_size + input_size, hidden_size)
        self.W_o = np.random.randn(hidden_size + input_size, hidden_size)
        self.W_y = np.random.randn(hidden_size, output_size)
        self.b_f = np.zeros((1, hidden_size))
        self.b_i = np.zeros((1, hidden_size))
        self.b_c = np.zeros((1, hidden_size))
        self.b_o = np.zeros((1, hidden_size))
        self.b_y = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def forward(self, inputs):
        h_prev = np.zeros((1, self.hidden_size))
        c_prev = np.zeros((1, self.hidden_size))
        for t in range(len(inputs)):
            x_t = inputs[t].reshape((1, self.input_size))
            concat = np.hstack((h_prev, x_t))
            f = self.sigmoid(np.dot(concat, self.W_f) + self.b_f)
            i = self.sigmoid(np.dot(concat, self.W_i) + self.b_i)
            c_bar = self.tanh(np.dot(concat, self.W_c) + self.b_c)
            c = f * c_prev + i * c_bar
            o = self.sigmoid(np.dot(concat, self.W_o) + self.b_o)
            h = o * self.tanh(c)
            h_prev, c_prev = h, c
        y = np.dot(h, self.W_y) + self.b_y
        return y

def lstm_model(data_history):
    input_size = 1
    hidden_size = 130
    output_size = 1
    model = LSTMModel(input_size, hidden_size, output_size)
    return model

data_history = 15  # number of past observations used
model_lstm = lstm_model(data_history)


In [26]:
# Model training process
training_history = []
for epoch in range(10):
    epoch_loss = 0
    for i in range(0, len(X_train), 32):
        X_batch = X_train[i:i+32].reshape(-1, 1)  # Reshape input data to match the model's input shape
        Y_batch = Y_train[i:i+32]
        preds = model_lstm.forward(X_batch)
        loss = np.mean((preds - Y_batch) ** 2)  # Mean squared error loss
        # Backpropagation (not implemented in this simplified example)
        # Update weights and biases (not implemented in this simplified example)
        epoch_loss += loss
    epoch_loss /= len(X_train) / 32  # Calculate average loss per batch
    # Evaluation on validation set
    val_preds = model_lstm.forward(X_val.reshape(-1, 1))  # Reshape validation data as well
    val_loss = np.mean((val_preds - Y_val) ** 2)
    print(f"Epoch {epoch+1}, Training Loss: {epoch_loss}, Validation Loss: {val_loss}")
    training_history.append((epoch_loss, val_loss))


Epoch 1, Training Loss: 22.914842464640405, Validation Loss: 31.294857864738407


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
def stats_reveal (training):
    #Plotting the validation and training curves of loss.
    plt.plot(training.history['loss'])
    plt.plot(training.history['val_loss'])


    #Setting the graph labels and tilte.
    plt.title("Model")
    plt.xlabel("Epoch")
    plt.ylabel("Loss Prediction")
    plt.legend(["Training Loss", "Validation Loss"])

    #Outputs the graph.
    plt.show()

#Summons the function to get a visualization on the curves loss.
stats_reveal (training)

In [ ]:
# activate the model's predictions on the test dataset
prognosis = model_lstm.predict(X_test)

plt.title("Model's prediction vs real prices")

# calculating the (RMSE) to determine accuracy of prediction

rmse = np.sqrt(np.mean(((prognosis - Y_test) ** 2)))
print("RSME: ", rmse)

#Plot : prediction data.
plt.plot(prognosis*factor_scale[20] + shift_info[20])

#Plots : real data.
plt.plot(Y_test*factor_scale[20] + shift_info[20])

#Output graph
plt.legend(["Predictions","Real data"])
plt.show()